![image info](https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2023/main/images/banner_1.png)

# Proyecto 1 - Predicción de precios de vehículos usados

En este proyecto podrán poner en práctica sus conocimientos sobre modelos predictivos basados en árboles y ensambles, y sobre la disponibilización de modelos. Para su desasrrollo tengan en cuenta las instrucciones dadas en la "Guía del proyecto 1: Predicción de precios de vehículos usados".

**Entrega**: La entrega del proyecto deberán realizarla durante la semana 4. Sin embargo, es importante que avancen en la semana 3 en el modelado del problema y en parte del informe, tal y como se les indicó en la guía.

Para hacer la entrega, deberán adjuntar el informe autocontenido en PDF a la actividad de entrega del proyecto que encontrarán en la semana 4, y subir el archivo de predicciones a la [competencia de Kaggle](https://www.kaggle.com/t/b8be43cf89c540bfaf3831f2c8506614).

## Datos para la predicción de precios de vehículos usados

En este proyecto se usará el conjunto de datos de Car Listings de Kaggle, donde cada observación representa el precio de un automóvil teniendo en cuenta distintas variables como: año, marca, modelo, entre otras. El objetivo es predecir el precio del automóvil. Para más detalles puede visitar el siguiente enlace: [datos](https://www.kaggle.com/jpayne/852k-used-car-listings).

## Ejemplo predicción conjunto de test para envío a Kaggle

En esta sección encontrarán el formato en el que deben guardar los resultados de la predicción para que puedan subirlos a la competencia en Kaggle.

In [72]:
import warnings
warnings.filterwarnings('ignore')

In [73]:
# Importación librerías

#Python
import pandas as pd
import numpy as np

#Regressor Models
from xgboost import XGBRegressor

# data processing
from sklearn import metrics
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    r2_score
)


# sklearn
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV, train_test_split
from sklearn.feature_selection import RFE
from sklearn.preprocessing import OneHotEncoder, power_transform, LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

In [74]:
# Carga de datos de archivo .csv
dataTraining = pd.read_csv('https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2023/main/datasets/dataTrain_carListings.zip')
dataTesting = pd.read_csv('https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2023/main/datasets/dataTest_carListings.zip', index_col=0)

In [75]:
# Visualización datos de entrenamiento
dataTraining.head()

,Price,Year,Mileage,State,Make,Model
0,34995,2017,9913,FL,Jeep,Wrangler
1,37895,2015,20578,OH,Chevrolet,Tahoe4WD
2,18430,2012,83716,TX,BMW,X5AWD
3,24681,2014,28729,OH,Cadillac,SRXLuxury
4,26998,2013,64032,CO,Jeep,Wrangler


In [76]:
# Visualización datos de test
dataTesting.head()

,Year,Mileage,State,Make,Model
ID,,,,,
0,2014,31909,MD,Nissan,MuranoAWD
1,2017,5362,FL,Jeep,Wrangler
2,2014,50300,OH,Ford,FlexLimited
3,2004,132160,WA,BMW,5
4,2015,25226,MA,Jeep,Grand


### Encoding

In [77]:
label_encoder = LabelEncoder()

# Aplica label encoding a la columna 'State'
dataTraining['State'] = label_encoder.fit_transform(dataTraining['State'])

# Aplica label encoding a las columnas 'Make' y 'Model'
dataTraining['Make'] = label_encoder.fit_transform(dataTraining['Make'])
dataTraining['Model'] = label_encoder.fit_transform(dataTraining['Model'])


In [78]:
X=dataTraining[['Year', 'Mileage', 'State', 'Make', 'Model']]
y=dataTraining['Price']

### Separacion Train/Test

In [79]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [80]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train[['Year', 'Mileage']])
X_test_scaled = scaler.transform(X_test[['Year', 'Mileage']])

### Modelo XGBRegressor

In [81]:
xgb_reg = XGBRegressor(objective='reg:squarederror')

In [82]:
xgb_reg.fit(X_train_scaled, y_train)
y_pred=xgb_reg.predict(X_test_scaled)
XGb_sin_mse=mean_squared_error(y_pred, y_test.values)
XGb_sin_r2=r2_score(y_pred, y_test.values)
XGb_sin_mae=mean_absolute_error(y_pred, y_test.values)

print(f'RMSE: {XGb_sin_mse**1/2}, R2 Score: {XGb_sin_r2}, MAE: {XGb_sin_mae}')

RMSE: 40161329.48271683, R2 Score: -1.257330776257389, MAE: 6915.933874752161


### Calibracion Modelo XGBRegressor

In [87]:
from sklearn.model_selection import RandomizedSearchCV

# Definir los hiperparámetros a explorar
param_grid = {
    'learning_rate': [0.01, 0.1, 0.3],
    'max_depth': [3, 5, 7],
    'n_estimators': [100, 200, 300]
}

xgb_reg_cal = XGBRegressor()

random_search = RandomizedSearchCV(xgb_reg_cal, param_distributions=param_grid, n_iter=50, scoring='neg_mean_squared_error', cv=10, verbose=2, random_state=42, n_jobs=-1)

random_search.fit(X_train_scaled, y_train)


best_params_rs = random_search.best_params_
print("Mejores hiperparámetros:", best_params_rs)

best_model = random_search.best_estimator_


Fitting 10 folds for each of 27 candidates, totalling 270 fits
Mejores hiperparámetros: {'n_estimators': 300, 'max_depth': 3, 'learning_rate': 0.1}


In [88]:
y_pred = best_model.predict(X_test_scaled)
MSEXGB=mean_squared_error(y_test, y_pred)
r2XGB=r2_score(y_test, y_pred)
MAEXGB=mean_absolute_error(y_test, y_pred)

print(f'RMSE: {MSEXGB**1/2}, R2 Score: {r2XGB}, MAE: {MAEXGB}')

RMSE: 40016286.55899231, R2 Score: 0.30436269176415165, MAE: 6910.2168050796045


In [89]:
# Predicción del conjunto de test - acá se genera un número aleatorio como ejemplo
np.random.seed(42)
y_pred = pd.DataFrame(np.random.rand(dataTesting.shape[0]) * 75000 + 5000, index=dataTesting.index, columns=['Price'])

In [90]:
# Guardar predicciones en formato exigido en la competencia de kaggle
y_pred.to_csv('test_submission.csv', index_label='ID')
y_pred.head()

,Price
ID,
0,33090.508914
1,76303.572981
2,59899.545636
3,49899.386315
4,16701.398033
